In [ ]:
# pip install requests fhirclient jupyter

In [67]:
# week2_fhir_patient_tool.py
"""
Week 2 – FHIR JSON & REST API (Patients)
- Search / save / summarize Patient resources using requests
- Supports search parameters (name, family, birthdate, _sort, _count)
- Can also save results as CSV
- (Optional) Includes example function using fhirclient

Usage examples:
  python week2_fhir_patient_tool.py --count 5 --sort -_lastUpdated
  python week2_fhir_patient_tool.py --name Jenny --birthdate 1982-04-07 --count 5
  python week2_fhir_patient_tool.py --family Kim --count 3 --out json
"""

import argparse
import json
from pathlib import Path
from typing import Dict, Any, List

import requests

# Base URL for Patient resources on public HAPI FHIR server
BASE = "https://hapi.fhir.org/baseR4/Patient"

# Output directory for saving results
OUT_DIR = Path("./FHIR/week2")
OUT_DIR.mkdir(parents=True, exist_ok=True)

def search_patients(params: Dict[str, Any]) -> Dict[str, Any]:
    """Search patients from the FHIR server with given parameters."""
    resp = requests.get(BASE, params=params, timeout=30)
    resp.raise_for_status()
    return resp.json()

def bundle_to_rows(bundle: Dict[str, Any]) -> List[Dict[str, Any]]:
    """Convert a FHIR Bundle of Patient resources into a list of dictionaries."""
    rows = []
    for e in bundle.get("entry", []):
        r = e.get("resource", {})
        if r.get("resourceType") != "Patient":
            continue
        rid = r.get("id", "")
        meta = r.get("meta", {})
        last = meta.get("lastUpdated", "")
        # Combine name fields
        name = "Unknown"
        names = r.get("name", [])
        if names:
            given = " ".join(names[0].get("given", [])).strip()
            family = (names[0].get("family") or "").strip()
            full = (given + " " + family).strip()
            if full:
                name = full
        rows.append({
            "id": rid,
            "Name": name,
            "Gender": r.get("gender", "Unknown"),
            "BirthDate": r.get("birthDate", "Unknown"),
            "LastUpdated": last
        })
    return rows

def save_raw_json(bundle: Dict[str, Any], path: Path):
    """Save the raw FHIR Bundle as formatted JSON."""
    path.write_text(json.dumps(bundle, indent=2, ensure_ascii=False), encoding="utf-8")

def save_csv(rows: List[Dict[str, Any]], path: Path):
    """Save the patient rows as a simple CSV file."""
    if not rows:
        path.write_text("", encoding="utf-8")
        return
    headers = list(rows[0].keys())
    lines = [",".join(headers)]
    for r in rows:
        # Replace commas inside values to avoid breaking CSV format
        vals = [str(r.get(h, "")).replace(",", " ") for h in headers]
        lines.append(",".join(vals))
    path.write_text("\n".join(lines), encoding="utf-8")

def print_table(rows: List[Dict[str, Any]], max_rows: int = 20):
    """Print the patient rows in a formatted table."""
    if not rows:
        print("No Patient resources in the bundle.")
        return
    print("id | Name | Gender | BirthDate | LastUpdated")
    print("-"*80)
    for r in rows[:max_rows]:
        print(f"{r['id']} | {r['Name']} | {r['Gender']} | {r['BirthDate']} | {r['LastUpdated']}")

def main():
    """Main CLI entry point for the patient search tool."""
    ap = argparse.ArgumentParser(description="Week2 – FHIR Patient search tool")
    ap.add_argument("--name", type=str, help="name contains")
    ap.add_argument("--family", type=str, help="family (surname) contains")
    ap.add_argument("--given", type=str, help="given name contains")
    ap.add_argument("--birthdate", type=str, help='e.g. 1982-04-07 or ge1980-01-01')
    ap.add_argument("--count", type=int, default=5, help="_count (default 5)")
    ap.add_argument("--sort", type=str, default="-_lastUpdated", help="sort, e.g. -_lastUpdated")
    ap.add_argument("--out", choices=["json", "csv", "none"], default="json", help="output format")
    args = ap.parse_args()

    # Build search parameters dictionary
    params = {"_count": args.count, "_sort": args.sort}
    if args.name: params["name"] = args.name
    if args.family: params["family"] = args.family
    if args.given: params["given"] = args.given
    if args.birthdate: params["birthdate"] = args.birthdate

    print("→ Searching with params:", params)
    bundle = search_patients(params)
    rows = bundle_to_rows(bundle)
    print_table(rows)

    # Save results
    if args.out != "none":
        if args.out == "json":
            raw_path = OUT_DIR / "patient_api.json"
            save_raw_json(bundle, raw_path)
            print(f"\nSaved raw JSON to: {raw_path}")
        elif args.out == "csv":
            csv_path = OUT_DIR / "patient_api.csv"
            save_csv(rows, csv_path)
            print(f"\nSaved CSV to: {csv_path}")

# (Optional) Example using fhirclient
def demo_fhirclient():
    """
    Example usage with `fhirclient` library.
    Install with:
        pip install fhirclient
    """
    try:
        from fhirclient import client
        from fhirclient.models import patient as fhir_patient
    except Exception:
        print("Install fhirclient first: pip install fhirclient")
        return
    settings = {'app_id': 'my_app', 'api_base': 'https://hapi.fhir.org/baseR4'}
    smart = client.FHIRClient(settings=settings)
    patients = fhir_patient.Patient.where(struct={'_count': 3}).perform_resources(smart.server)
    for p in patients:
        name = "Unknown"
        if p.name and len(p.name) > 0:
            g = p.name[0].given or []
            fam = p.name[0].family or ""
            name = (" ".join(g) + " " + fam).strip() or "Unknown"
        gender = p.gender or "Unknown"
        bdate = p.birthDate.isostring if getattr(p, "birthDate", None) else "Unknown"
        print(f"{name} | {gender} | {bdate}")

if __name__ == "__main__":
    main()
    # demo_fhirclient()  # Uncomment if you want to run the fhirclient example


usage: ipykernel_launcher.py [-h] [--name NAME] [--family FAMILY]
                             [--given GIVEN] [--birthdate BIRTHDATE]
                             [--count COUNT] [--sort SORT]
                             [--out {json,csv,none}]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/cocoxoxo/Library/Jupyter/runtime/kernel-90c99b1e-0fd3-45a7-a341-fb8150a6239d.json


SystemExit: 2

/opt/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [69]:
import sys
sys.argv = ['week2_fhir_patient_tool.py', '--name', 'Jenny', '--count', '5']
%run week2_fhir_patient_tool.py


Exception: File `'week2_fhir_patient_tool.py'` not found.